In [1]:
import os
import sys
import numpy as np
import pandas as pd

In [2]:
src = open("./archive/nana/train.src", 'r', encoding='utf-8')
tgt = open("./archive/nana/train.tgt", 'r', encoding='utf-8')
nations = dict()
count = np.zeros(1000)
num_nation = 0
info = []
name_list = []
for l1, l2 in zip(src, tgt):
    names = l1[:-1].lower().replace(",", "").split(" ")
    nation = l2[:-1].lower()
    if nation not in nations.keys():
        nations[nation] = num_nation
        num_nation += 1 
    for name in names:
        info.append((name, nations[nation]))
    count[nations[nation]] += 1

src.close()
tgt.close()


In [3]:
file_list = os.listdir("./nationality-classify-master/datasets")
for fname in file_list:
    nation = fname.replace(".txt", "")
    if nation not in nations.keys():
        nations[nation] = num_nation
        num_nation += 1
    count[nations[nation]] += 1
    with open(os.path.join("./nationality-classify-master/datasets", fname), 'r', encoding='utf-8') as f:
        for l in f:
            names = l1[:-1].lower().split(" ")
            info.append((name, nations[nation]))


In [3]:
sort_nations = sorted(nations.keys())
info_set = set(info)
data = open("db2.txt", 'w', encoding='utf-8')
for name, nid in info_set:
    if len(name) >= 3 and name.isalpha():
        data.write("%s %d\n"%(name, nid))

with open("nations.txt", 'w', encoding='utf-8') as f:
    for nation in nations.keys():
        f.write("%s\n"%(nation))

data.close()


In [14]:
names = []
with open("db2.txt", 'r', encoding='utf-8') as f:
    for line in f:
        name = line[:-1].split(",")[0]
        names.append(name)

In [6]:
na2re = []
regions = dict()
with open("nation-region.txt", 'r', encoding='utf-8') as f:
    num = 0
    for line in f:
        region = line[:-1].split(',')[1]
        if region not in regions.keys():
            regions[region] = num
            num += 1
        na2re.append(regions[region])

In [ ]:
 count[nations[nation]]

In [7]:
db = open("db.txt", 'r', encoding='utf-8')
redb = open("redb.txt", 'w', encoding='utf-8')

recount = np.zeros(13)

for line in db:
    name, nation = line[:-1].split(" ")
    re = na2re[int(nation)]
    recount[re] += 1
    redb.write("%s %d\n"%(name, re))

db.close()
redb.close()

In [10]:
with open("regions.txt", 'w', encoding='utf-8') as f:
    for region, rid in regions.items():
        print(rid)
        f.write("%s %d\n"%(region, recount[rid]))

0
1
2
3
4
5
6
7
8
9
10
11
12
